In [10]:
!pip install kagglehub
!pip install contractions
!pip install num2words
!pip install inflect
!pip install jieba
!pip install cn2an
!pip install sacrebleu
!pip install safetensors
!pip install datasets

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import unicodedata
import contractions
import inflect

import num2words
import cn2an
import jieba

import re
from nltk.corpus import stopwords
from num2words import num2words

import pandas as pd

from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from datasets import load_dataset

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from safetensors.torch import load_file

model_name = 'utrobinmv/t5_translate_en_ru_zh_small_1024'
weights_path = '/content/drive/MyDrive/Colab Notebooks/model_weights/model.safetensors'
state_dict = load_file(weights_path)

model = T5ForConditionalGeneration.from_pretrained(model_name,state_dict=state_dict)
tokenizer = T5Tokenizer.from_pretrained(model_name)

prefix = 'translate to zh: '
src_text = prefix + "1. Яблоко."

# translate Russian to Chinese
input_ids = tokenizer(src_text, return_tensors="pt")

generated_tokens = model.generate(**input_ids)

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result)
#开发的目的是为用户提供个人同步翻译。

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/809 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/221M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

['一.苹果。']


In [14]:
import re
import jieba

class CleanChineseText:
    def __init__(self, zh_text):
        self.zh_text = zh_text
    def escape_en_char(self):
        en_pattern = "[a-zA-Z]+"
        self.zh_text = re.sub(en_pattern, ' ', self.zh_text)
        return self
    def remove_enter_and_symbols(self):
        self.zh_text = re.sub(r'^\d+\.', '', self.zh_text).strip()
        return self
    def remove_gaps(self):
        self.zh_text = self.zh_text.replace(' ', '')
        return self
    def remove_punct(self):
        self.zh_text = self.zh_text.strip('，')
        return self

    def clean_all(self):
        self.remove_punct()
        self.remove_enter_and_symbols()
        self.escape_en_char()
        self.remove_gaps()
        return self.zh_text

In [15]:
import re
class CleanRussianText:
    def __init__(self, ru_text):
        self.ru_text = ru_text
    # def to_one_style(self):
    #     # Lower case
    #     self.ru_text = self.ru_text.lower()
    #     return self
    # def remove_numbers_enters_symbols(self):
    #     # remove all symbols, numbers and /n in the end of the strings
    #     self.ru_text = re.sub(r'[^\w\s]', '', self.ru_text).strip()
    #     # self.ru_text = re.sub(r'^\d+\.', '', self.ru_text).strip()
    #     # self.ru_text = re.sub('[^а-яА-Я0-9\s]', ' ', self.ru_text)
    #     return self

    # def combine_numbers(self):
    #     # combine numbers with a space, such as '40 000' to '40000'
    #     self.ru_text = re.sub(r'(\d+)\s*(\d+)', r'\1\2', self.ru_text)
    #     return self
    def separate_numbers_and_words(self):
        # separate words like '11летнему' to '11 летнему'
        self.ru_text = re.sub(r'(\d+)([а-яА-Я]+)', r'\1 \2', self.ru_text)
        return self

    # def numbers_to_words(self):
    #     # convert numbers to words '40000' to 'сорок тысяч'
    #     def convert_numbers(match):
    #       return num2words(int(match.group()), lang='ru', to='ordinal')
    #     self.ru_text = re.sub(r'\d+', convert_numbers, self.ru_text)
    #     return self

    def clean_all(self):
      self.ru_text = self.ru_text.lower().strip()
      self.ru_text = re.sub(r'\s+', ' ', self.ru_text)
      self.ru_text = re.sub(r'[^\w\s.,!?;:-]', '', self.ru_text)
      self.ru_text = re.sub(r'(\d+)\s*(\d+)', r'\1\2', self.ru_text)
      # self.combine_numbers()
      self.separate_numbers_and_words()
      # self.numbers_to_words()
      return self.ru_text

In [16]:
text = ' ♫ мы кое-что ищем, а, 11летний ♫'
cleaner = CleanRussianText(text)
result = cleaner.clean_all()
print(result)

 мы кое-что ищем, а, 11 летний 


In [17]:
text = '，我是1个anzh很好好的15岁的孩子，'
cleaner = CleanChineseText(text)
result = cleaner.clean_all()
print(result)

我是1个很好好的15岁的孩子


In [18]:
def tokenize_dataset(source_ru, target_zh, tokenizer=tokenizer):
    prefix = 'translate to zh'
    source_ru_texts = [f'{prefix}: {text}' for text in source_ru]
    tokenized_ru_texts = [tokenizer(text, truncation=True, padding=True, return_tensors='pt') for text in source_ru_texts]

    target_zh_texts = [tokenizer(zh_text, truncation=True, padding=True, return_tensors='pt') for zh_text in target_zh]
    return tokenized_ru_texts, target_zh_texts

class TokenizedT5CustomDataset(Dataset):
    def __init__(self, tokenized_ru_texts_ids, target_zh_texts_ids):
        self.source_ru = tokenized_ru_texts_ids
        self.target_zh = target_zh_texts_ids

    def __getitem__(self, idx):
        return {
            'input_ids': self.source_ru[idx]['input_ids'].squeeze(0),
            'attention_mask': self.source_ru[idx]['attention_mask'].squeeze(0),
            'labels': self.target_zh[idx]['input_ids'].squeeze(0)
        }
    def __len__(self):
        return len(self.source_ru)

In [19]:
subtitle_dataset = load_dataset("open_subtitles", lang1="ru", lang2="zh_cn", trust_remote_code=True)

README.md:   0%|          | 0.00/7.45k [00:00<?, ?B/s]

open_subtitles.py:   0%|          | 0.00/6.22k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
df = pd.DataFrame(subtitle_dataset['train'][:1000000])

In [21]:
df['cleaned_ru'] = df['translation'].apply(lambda x: CleanRussianText(x['ru']).clean_all())

In [22]:
# df['translation'][2185946]

In [23]:
df['cleaned_zh'] = df['translation'].apply(lambda x: CleanChineseText(x['zh_cn']).clean_all())

In [24]:
df['translation'][999999]

{'ru': 'Хорошо сделано, Боф, но излишне, совершенно излишне.',
 'zh_cn': '做得好，鲍夫 不过你这是多此一举'}

In [25]:
from sklearn.model_selection import train_test_split
source_ru_train_subt, source_ru_val_subt, target_zh_train_subt, target_zh_val_subt = train_test_split(
    df['cleaned_ru'], df['cleaned_zh'], test_size=0.2, random_state=42)

In [26]:
tokenized_ru_texts_train, target_zh_texts_train = tokenize_dataset(source_ru_train_subt, target_zh_train_subt)
tokenized_ru_texts_val, target_zh_texts_val = tokenize_dataset(source_ru_val_subt, target_zh_val_subt)

In [27]:
print(tokenized_ru_texts_train[:10])

[{'input_ids': tensor([[21809,    19,     4,  3631,    31,    35,  1079,   803,   304,     5,
             1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[21809,    19,     4,  3631,    31,  1277,    51,   206,  5006,     5,
            14,   477,    34,  2352,    70,   151,  4296,  2147,   354,  3057,
             3,    52,  1582,   248,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}, {'input_ids': tensor([[21809,    19,     4,  3631,    31,  9756,  3622,  2341,  2232,     3,
           187, 31016,   257, 54897, 12275,    99,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[21809,    19,     4,  3631,    31,   407,    33,   756,   159,  3877,
            33, 42305,    14, 28273,     3,   314,  1738,     4, 25195,  5807,
            33,  5891,    32,   184,     5,     1]]), 'attention_mask': tensor([[

In [28]:
# target_zh_texts_train

In [29]:
print(len(target_zh_texts_train))

800000


In [30]:
print(tokenized_ru_texts_val[:10])

[{'input_ids': tensor([[21809,    19,     4,  3631,    31,    16, 13431,     5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[21809,    19,     4,  3631,    31,    35,   160,    16,  8678, 13330,
            34,  1765,  9664,     5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[21809,    19,     4,  3631,    31,     4,    93,    55,    72,   227,
          4464,   284,   136,  2392,  2590,   233,   284,   128,   617,  1558,
             3,  1710,  5512,   396,     5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}, {'input_ids': tensor([[21809,    19,     4,  3631,    31,   109, 10997,     3,   187,   181,
            51,   215, 16977,   947,  1202,     3,    16,   181, 31748,     5,
             1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, {'input

In [31]:
print(target_zh_texts_val[:10])

[{'input_ids': tensor([[   4,  452, 5546,    1]]), 'attention_mask': tensor([[1, 1, 1, 1]])}, {'input_ids': tensor([[ 3558,  2820,  1036,    99,   100,    29,     3, 11503, 16063,   100,
         39530,   452,  1300,   913, 10004,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[3558, 1566,  653,  583, 5488,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[14787, 28161, 43445,   259,   934,     3, 35022, 15692,   868,   107,
             1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[ 2651, 22122,   964,  3411,   316,    59,  3292, 31831,  2032,     3,
             1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[    4, 10530,  2985, 39806,   552,  1036,  5546,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[    4,  3640, 25099,   107,  2061,  5711,     1]]), 'atte

In [32]:
target_zh_texts_val[1]

{'input_ids': tensor([[ 3558,  2820,  1036,    99,   100,    29,     3, 11503, 16063,   100,
         39530,   452,  1300,   913, 10004,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [33]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

train_dataset = TokenizedT5CustomDataset(tokenized_ru_texts_ids=target_zh_texts_train[:1000000], target_zh_texts_ids=target_zh_texts_train[:100000])
test_dataset = TokenizedT5CustomDataset(tokenized_ru_texts_ids=tokenized_ru_texts_val[:1000000], target_zh_texts_ids=target_zh_texts_val[:100000])

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=False, collate_fn=data_collator)


In [34]:
test_dataset_test, test_dataset_val = train_test_split(test_dataset, test_size=0.5, random_state=42)

In [35]:
len(test_dataset_test)

50000

In [36]:
len(test_dataset_test) + len(test_dataset_val)

100000

In [37]:
test_dataset_val[0]

{'input_ids': tensor([21809,    19,     4,  3631,    31,   201,   314, 17263,   184,     3,
           314,  7376,  2089,   279,     3,    52,   770,  1582, 42850,    81,
           490,     5,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor([    4, 11433,  1150, 11433,  3592,     3,  3326,  3326,  5546,     1])}

In [38]:
test_loader = DataLoader(dataset=test_dataset_test, batch_size=128, shuffle=False, collate_fn=data_collator)
dataset_val_loader = DataLoader(dataset=test_dataset_val, batch_size=128, shuffle=False, collate_fn=data_collator)

In [39]:
from nltk.translate.bleu_score import corpus_bleu
def bleu_score(model, tokenizer, eval_pred_loader):
  actual, predicted = [], []
  model.to(device)
  model.eval()
  with torch.no_grad():

    for batch in eval_pred_loader:
      # input_ids = batch['input_ids']
      # attention_mask = batch['attention_mask']
      # labels = batch['labels']
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

      decoded_labels = tokenizer.batch_decode(
          [[token if token != -100 else tokenizer.pad_token_id for token in label] for label in labels],
          skip_special_tokens=True,
      )

      predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

      actual.append([decoded_labels[0]])
      predicted.append(predictions[0])

  bleu_dic = {}
  bleu_dic['bleu'] = corpus_bleu(actual, predicted)
  # bleu_dic['1-2-grams'] = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
  # bleu_dic['1-3-grams'] = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0), smoothing_function=smoothing)
  # bleu_dic['1-4-grams'] = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)

  return bleu_dic

In [40]:
# model.to(device)

In [41]:
# import torch
# bleu_test_subt = bleu_score(model, tokenizer, test_loader)
# print(bleu_test_subt)

In [42]:
from transformers import Trainer
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='/result',
    evaluation_strategy='steps',
    per_device_train_batch_size=512,
    per_device_eval_batch_size=128,
    gradient_checkpointing=True,
    gradient_accumulation_steps=2,
    learning_rate=5e-05,
    num_train_epochs=20,
    logging_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    seed=42,
    logging_dir="./logs",
    fp16=True,
    report_to="none"

)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-42-5e95e0a728b8>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [43]:
import torch
torch.cuda.empty_cache()

In [44]:
torch.cuda.set_per_process_memory_fraction(0.98, device=0)  # Adjust as needed

In [45]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [46]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
1000,0.306000,8.875756
2000,0.048300,9.920909
3000,0.028800,10.477384


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


TrainOutput(global_step=3900, training_loss=0.10353266006860977, metrics={'train_runtime': 1572.5894, 'train_samples_per_second': 1271.788, 'train_steps_per_second': 2.48, 'total_flos': 3.0565235821510656e+16, 'train_loss': 0.10353266006860977, 'epoch': 19.900255754475705})

In [55]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/model_weights/new/')
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/model_weights/new/')

('/content/drive/MyDrive/Colab Notebooks/model_weights/new/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/model_weights/new/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/model_weights/new/spiece.model',
 '/content/drive/MyDrive/Colab Notebooks/model_weights/new/added_tokens.json')

In [54]:
text= '''
Сдаётся уютная 1-комнатная квартира с хорошим ремонтом (использовались только качественные строительные материалы), в доме "Токио ", квартал Азия, на 8 этаже 24 этажного
жилого дома. Отличное месторасположение дома – недалеко находится супермаркет, школа, сад, пиццерия, кофейня, большая парковочная площадка. На входе комфортабельного 24 -этажного дома «Токио», жильцов встречают дизайнерские лобби с зоной отдыха, местом для консьержа, и санитарной комнатой, установлены бесшумные
лифты OTIS.В квартире сделан хороший ремонт: использовалась декоративная штукатурка «Prima Vera»,  смеситель, душ в ванной, краны и сантехника «GROHE».
Панорамные окна в квартире и на лоджии. Лоджия застеклена. Входная дверь-металлическая, с хорошей шумоизоляцией и зеркалом. Полы: керамическая плитка.
Встроенная кухня на заказ «EGGER» со встроенной бытовой техникой «ZORG»:
духовым шкафом, индукционной плитой, печью СВЧ и встроенным холодильником.
Вся техника находится на гарантийном обслуживании;
В прихожей установлен шкаф-купе на заказ «EGGER, с зеркалом.
Установлена ванная, умывальник, качественная сантехника, стиральная машина «BOSCH».
Спальное место и телевизор устанавливаем по запросу арендатора.
Отличное транспортное сообщение: быстро и легко можно добраться как до центра города,
так и до МКАД. В 2024 году планируется открытие ст. метро «Аэродромная».
Квартира готова к проживанию. Оплата коммунальных платежей, собственник зарегистрирован.
'''

model.eval()

prefix = 'translate to zh: '
src_text = prefix + text

# translate Russian to Chinese
input_ids = tokenizer(src_text, return_tensors="pt", truncation=True, max_length=500).to('cuda')

generated_tokens = model.generate(**input_ids)

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result)

['给舒服1-卧室公寓与好的修理(只使用了优质建筑材料),在房子"东京",街区亚洲。建门-宾馆设有浴室,学校,园,烤箱,大停车场。']
